In [ ]:
import numpy as np
import pandas as pd
import re

# data cleaning : for replacing na
from sklearn.impute import SimpleImputer

# text data cleaning :
import nltk
nltk.download('stopwords')

# data exploration :
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing :
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# models
from sklearn.svm import LinearSVC

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adjal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Read data :

In [78]:
# Read the CSV file into a Pandas DataFrame
dataset_raw = pd.read_csv("X_train_Hi5.csv",nrows=10000)

# view 5 first raws
dataset_raw.head()

,row_index,piezo_station_department_code,piezo_station_update_date,piezo_station_investigation_depth,piezo_station_department_name,piezo_station_commune_code_insee,piezo_station_pe_label,piezo_station_bdlisa_codes,piezo_station_altitude,piezo_station_bss_code,...,prelev_longitude_2,prelev_latitude_2,prelev_commune_code_insee_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const,piezo_groundwater_level_category
0,0,01,Sun Jul 14 13:00:02 CEST 2024,20.00,Ain,01073,PIEZOMETRE - MARAIS DE LAVOURS (CEYZERIEU - BR...,['712AH37'],232.00,07004X0046/D6-20,...,5.698947,45.725106,1454.0,1.793055e+09,11.8,992.0,25250,2.9,16.2,High
1,1,01,Sun Jul 14 13:00:02 CEST 2024,35.60,Ain,01363,PIEZOMETRE - GRAVIERE (ST-JEAN-LE-VIEUX - BRGM...,['712GB05'],247.25,06754X0077/F1,...,5.464933,46.210734,1051.0,1.085125e+09,0.6,1786.0,24660,44.5,11,Very High
2,2,01,Sun Jul 14 13:00:02 CEST 2024,35.22,Ain,01244,PIEZOMETRE - BORD AUTOROUTE (MEXIMIEUX - BRGM ...,['040AJ43'],218.77,06993X0226/MEXI_2,...,5.085060,45.812828,69266.0,3.810492e+08,0,8085.0,24890,8.4,7.8,High
3,3,01,Sun Jul 14 13:00:02 CEST 2024,34.20,Ain,01288,PIEZOMETRE - GRENY (PERON - BRGM 01) - BSH,"['516AA00', '516AF00']",499.85,06533X0070/F2,...,5.802841,46.366049,39286.0,3.800911e+08,1.5,2838.0,39700,2.4,5.2,Very High
4,4,01,Sun Jul 14 13:00:02 CEST 2024,37.30,Ain,01422,FORAGE - ENCLOS (TOSSIAT - BRGM 01) - BSH,['507AB00'],260.00,06518X0026/P2,...,5.377265,46.080989,1273.0,1.966631e+07,0.2,1352.0,26180,21.5,9.8,Very Low


In [66]:
# Get general information on the dataset (ncol, nrow, nbr missing values, dtypes)
dataset_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830316 entries, 0 to 2830315
Columns: 136 entries, row_index to piezo_groundwater_level_category
dtypes: float64(90), int64(6), object(40)
memory usage: 2.9+ GB


In [67]:
# Number of missing values for each variable
dataset_raw.isna().sum()

row_index                                 0
piezo_station_department_code             0
piezo_station_update_date                 0
piezo_station_investigation_depth    179062
piezo_station_department_name             0
                                      ...  
insee_pop_commune                     26820
insee_med_living_level                26820
insee_%_ind                           26820
insee_%_const                         26820
piezo_groundwater_level_category          0
Length: 136, dtype: int64

# Target encoding :

In [68]:
# Target view
target__value_count = dataset_raw['piezo_groundwater_level_category'].value_counts()

print(target__value_count)

piezo_groundwater_level_category
Average      649464
Low          629911
High         582020
Very Low     537077
Very High    431844
Name: count, dtype: int64


In [69]:
# Encode target : piezo_groundwater_level_category


# Define your custom mapping
mapping = {
    'Very Low': 0,
    'Low': 1,
    'Average': 2,
    'High': 3,
    'Very High': 4
}

# Apply the mapping to the column
dataset_raw['piezo_groundwater_level_category_encoded'] = dataset_raw['piezo_groundwater_level_category'].replace(mapping)

In [70]:
# Target view
target__value_count_encoded = dataset_raw['piezo_groundwater_level_category_encoded'].value_counts()

print(target__value_count_encoded)

piezo_groundwater_level_category_encoded
2    649464
1    629911
3    582020
0    537077
4    431844
Name: count, dtype: int64


# Date time collumns :

In [71]:
# Reducing the dataset for faster analysing

# List of columns to keep
columns_to_keep = ['piezo_station_update_date', 'piezo_measurement_date', 'hydro_observation_date_elab','meteo_date']

# Select only the columns to keep
dataset = dataset_raw[columns_to_keep]

# view
dataset.head()

,piezo_station_update_date,piezo_measurement_date,hydro_observation_date_elab,meteo_date
0,Sun Jul 14 13:00:02 CEST 2024,2020-01-01,2020-01-01,2020-01-01
1,Sun Jul 14 13:00:02 CEST 2024,2020-01-01,2020-01-01,2020-01-01
2,Sun Jul 14 13:00:02 CEST 2024,2020-01-01,2020-01-01,2020-01-01
3,Sun Jul 14 13:00:02 CEST 2024,2020-01-01,2020-01-01,2020-01-01
4,Sun Jul 14 13:00:02 CEST 2024,2020-01-01,2020-01-01,2020-01-01


In [72]:
# to_datetime

dataset['piezo_station_update_date'] = pd.to_datetime(dataset['piezo_station_update_date'])
dataset['piezo_measurement_date'] = pd.to_datetime(dataset['piezo_measurement_date'])
dataset['hydro_observation_date_elab'] = pd.to_datetime(dataset['hydro_observation_date_elab'])
dataset['meteo_date'] = pd.to_datetime(dataset['meteo_date'])

dataset.head()

,piezo_station_update_date,piezo_measurement_date,hydro_observation_date_elab,meteo_date
0,2024-07-14 13:00:02,2020-01-01,2020-01-01,2020-01-01
1,2024-07-14 13:00:02,2020-01-01,2020-01-01,2020-01-01
2,2024-07-14 13:00:02,2020-01-01,2020-01-01,2020-01-01
3,2024-07-14 13:00:02,2020-01-01,2020-01-01,2020-01-01
4,2024-07-14 13:00:02,2020-01-01,2020-01-01,2020-01-01


# piezo_measurement_date preprocessing :

In [ ]:
are_identical = (dataset_raw['piezo_measurement_date'] == dataset_raw['hydro_observation_date_elab']).all()

print("Are all rows identical?", are_identical)

are_identical = (dataset_raw['piezo_measurement_date'] == dataset_raw['meteo_date']).all()

print("Are all rows identical?", are_identical)

Are all rows identical? True
Are all rows identical? True


In [ ]:
# drop identical variables :
dataset.drop(columns=["hydro_observation_date_elab","meteo_date"],inplace=True)

In [ ]:
# Extract the year
dataset['measurement_year'] = dataset['piezo_measurement_date'].dt.year

# Calculate the fraction of the year for mm-dd
dataset['measurement_day_of_year'] = dataset['piezo_measurement_date'].dt.day_of_year  # Day number within the year (1-365/366)
dataset['measurement_day_of_year_normalised'] = dataset['measurement_day_of_year'] / 366  # Normalize to [0, 1]

# view
dataset.head()

,piezo_station_update_date,piezo_measurement_date,measurement_year,measurement_day_of_year,measurement_day_of_year_normalised
0,2024-07-14 13:00:02,2020-01-01,2020,1,0.002732
1,2024-07-14 13:00:02,2020-01-01,2020,1,0.002732
2,2024-07-14 13:00:02,2020-01-01,2020,1,0.002732
3,2024-07-14 13:00:02,2020-01-01,2020,1,0.002732
4,2024-07-14 13:00:02,2020-01-01,2020,1,0.002732


In [ ]:
# Encode target : piezo_groundwater_level_category

# Define your custom mapping
mapping = {
    2020: 0,
    2021: 1,
    2022: 2,
    2023: 3
}

# Apply the mapping to the column
dataset['measurement_year_encoded'] = dataset['measurement_year'].replace(mapping)

# view
dataset.head()

,piezo_station_update_date,piezo_measurement_date,measurement_year,measurement_day_of_year,measurement_day_of_year_normalised,measurement_year_encoded
0,2024-07-14 13:00:02,2020-01-01,2020,1,0.002732,0
1,2024-07-14 13:00:02,2020-01-01,2020,1,0.002732,0
2,2024-07-14 13:00:02,2020-01-01,2020,1,0.002732,0
3,2024-07-14 13:00:02,2020-01-01,2020,1,0.002732,0
4,2024-07-14 13:00:02,2020-01-01,2020,1,0.002732,0


In [ ]:
# drop transformed collumns :
dataset.drop(columns=["measurement_year","measurement_day_of_year"],inplace=True)

# view
dataset.head()


,piezo_station_update_date,piezo_measurement_date,measurement_day_of_year_normalised,measurement_year_encoded
0,2024-07-14 13:00:02,2020-01-01,0.002732,0
1,2024-07-14 13:00:02,2020-01-01,0.002732,0
2,2024-07-14 13:00:02,2020-01-01,0.002732,0
3,2024-07-14 13:00:02,2020-01-01,0.002732,0
4,2024-07-14 13:00:02,2020-01-01,0.002732,0


# piezo_station_update_date :

In [ ]:
piezo_station_update_date__value_count = dataset['piezo_station_update_date'].value_counts()

print(piezo_station_update_date__value_count)

piezo_station_update_date
2024-06-28 07:31:38    2042184
2024-07-14 13:00:02      69484
2020-01-24 16:44:49      30933
2020-02-27 06:03:18      30151
2024-07-28 13:00:01      23335
                        ...   
2024-08-06 19:29:52        302
2023-12-11 19:54:54        147
2024-01-24 20:45:03        145
2024-10-02 07:03:30         36
2024-01-09 05:04:59         11
Name: count, Length: 476, dtype: int64


In [ ]:
dataset['is_dominant_date'] = dataset['piezo_station_update_date'] == '2024-06-28 07:31:38'  # Replace with the most common date
correlation = dataset['is_dominant_date'].corr(dataset_raw['piezo_groundwater_level_category_encoded'])
print("Correlation:", correlation)


Correlation: 0.04371680801245096


In [ ]:
# Calculate the month
dataset['piezo_station_update_date_month'] = dataset['piezo_station_update_date'].dt.month  # Day number within the year (1-365/366)

# Extract the year
dataset['piezo_station_update_date_year'] = dataset['piezo_station_update_date'].dt.year

# view
dataset.head()

,piezo_station_update_date,piezo_measurement_date,measurement_day_of_year_normalised,measurement_year_encoded,is_dominant_date,piezo_station_update_date_month,piezo_station_update_date_year
0,2024-07-14 13:00:02,2020-01-01,0.002732,0,False,7,2024
1,2024-07-14 13:00:02,2020-01-01,0.002732,0,False,7,2024
2,2024-07-14 13:00:02,2020-01-01,0.002732,0,False,7,2024
3,2024-07-14 13:00:02,2020-01-01,0.002732,0,False,7,2024
4,2024-07-14 13:00:02,2020-01-01,0.002732,0,False,7,2024


In [ ]:
# drop transformed collumns :
dataset.drop(columns=["piezo_station_update_date","piezo_measurement_date","is_dominant_date"],inplace=True)

# view
dataset.head()

,measurement_day_of_year_normalised,measurement_year_encoded,piezo_station_update_date_month,piezo_station_update_date_year
0,0.002732,0,7,2024
1,0.002732,0,7,2024
2,0.002732,0,7,2024
3,0.002732,0,7,2024
4,0.002732,0,7,2024


In [ ]:
piezo_station_update_date_year__value_count = dataset['piezo_station_update_date_year'].value_counts()

print(piezo_station_update_date_year__value_count)

piezo_station_update_date_year
2024    2710087
2020      65555
2023      25434
2022      18328
2021      10912
Name: count, dtype: int64


In [ ]:
# Encode piezo_station_update_date_year
mapping = {
    2020: 0,
    2021: 1,
    2022: 2,
    2023: 3,
    2024: 4
}

# Apply the mapping to the column
dataset['piezo_station_update_date_year'] = dataset['piezo_station_update_date_year'].replace(mapping)

# view
dataset.head()

,measurement_day_of_year_normalised,measurement_year_encoded,piezo_station_update_date_month,piezo_station_update_date_year
0,0.002732,0,7,4
1,0.002732,0,7,4
2,0.002732,0,7,4
3,0.002732,0,7,4
4,0.002732,0,7,4
